In [ ]:
import numpy as np  # np mean, np random
import pandas as pd  # read csv, df manipulation
import streamlit as st  # 🎈 data web app development
import os
import altair as alt  # Import Altair for plotting

st.set_page_config(
    page_title="Real-Time Data Science Dashboard",
    page_icon="✅",
    layout="wide",
)

df = pd.read_csv('Contabilidad Total - DB.csv')  # Name of your csv file

def daily_income_data(df):
    df['Check in'] = pd.to_datetime(df['Check in'], format='%d/%m/%Y')
    df['Check out'] = pd.to_datetime(df['Check out'], format='%d/%m/%Y')
    df['Income'] = pd.to_numeric(df['Income'], errors='coerce')
    daily_income_data = []
    for index, row in df.iterrows():
        if row['Check out'] > row['Check in']:
            num_nights = (row['Check out'] - row['Check in']).days
            if num_nights > 0:
                pro_rata_income = row['Income'] / num_nights

                current_date = row['Check in']
                while current_date < row['Check out']:
                    daily_income_data.append({
                        'Date': current_date,
                        'Income': pro_rata_income,
                        'Country': row['Country'],  # Adding 'Country' information
                        'Platform': row['Platform'],  # Adding 'Platform' information
                        'Apartment': row['Apartment'] # Adding 'Apartment' information
                    })
                    current_date += pd.DateOffset(days=1)
    daily_income = pd.DataFrame(daily_income_data)  
    return daily_income

dailyIncome = daily_income_data(df)

# Sidebar for filtering options
st.sidebar.header("Filter Options")

# Select Years
selected_years = st.sidebar.multiselect(
    "Select Years",
    dailyIncome['Date'].dt.year.unique(),
    default=[2023]
)

# Select Apartments
selected_apartments = st.sidebar.multiselect(
    "Select Apartment",
    dailyIncome['Apartment'].unique(),
    default=[dailyIncome['Apartment'].iloc[0]]
)

# Pre-select all unique platforms as default
all_platforms = dailyIncome['Platform'].unique()
select_all_platforms = st.sidebar.checkbox("Select All Platforms", value=True)

if select_all_platforms:
    selected_platforms = all_platforms
else:
    selected_platforms = st.sidebar.multiselect(
        "Select Platform",
        all_platforms,
        default=all_platforms
    )

# Map month numbers to month names
month_names = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
    7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
}
sorted_months = sorted(month_names.keys())  # Sort the month numbers

# Select Months
all_months_selected = st.sidebar.checkbox("Select All Months", value=True)
if all_months_selected:
    selected_months = sorted_months
else:
    selected_months = st.sidebar.multiselect(
        "Select Months",
        sorted_months,
        format_func=lambda x: month_names[x]
    )

# Filter the DataFrame based on user selections
filtered_df = dailyIncome[
    (dailyIncome['Date'].dt.year.isin(selected_years)) &
    (dailyIncome['Date'].dt.month.isin(selected_months)) &
    (dailyIncome['Apartment'].isin(selected_apartments)) &
    (dailyIncome['Platform'].isin(selected_platforms))
]

# Pre-process the data to aggregate incomes for each month
agg_df = filtered_df.groupby(['Date', 'Apartment'])['Income'].sum().reset_index()
# Create a custom tooltip
tooltip = ['Date:T', 'Income:Q', 'Apartment:N']

# Check if the aggregated DataFrame is not empty
if not agg_df.empty:
    #######
    #FIG 1#
    #######
    # Define tooltip configuration for interactive chart tooltips
    tooltip = [
        alt.Tooltip('Date:T', title='Date', format='%B %Y'),  # Display month and year in tooltip
        alt.Tooltip('Income:Q', title='Income', format=',.2f'),  # Format income with 2 decimal places
        alt.Tooltip('Apartment:N', title='Apartment')  # Display apartment name in tooltip
    ]

    # Create an Altair bar chart
    chart = alt.Chart(agg_df).mark_bar().encode(
        x=alt.X('Date:T', timeUnit='month', axis=alt.Axis(format='%b', title='Month')),  # Format x-axis to show month names
        y=alt.Y('Income:Q', title='Income'),  # Label y-axis as "Income"
        color=alt.Color('Apartment:N', legend=None),  # Use apartment names for color, no need for legend
        tooltip=tooltip  # Apply the defined tooltip configuration
    ).properties(
        width=600,  # Set chart width
        height=400,  # Set chart height
        title='Income by Month'  # Set chart title
    )
    
    #Main title 
    st.title("Canaries vacation")
    
    # Display the Altair chart using Streamlit, adapting to container width
    st.altair_chart(chart, use_container_width=True)
    
    #######
    #FIG 2#
    #######

    # Count occurrences of each country
    country_counts = filtered_df['Country'].value_counts()
    # Create a new DataFrame from the country_counts Series
    counts_df = pd.DataFrame({'Country': country_counts.index, 'Count': country_counts.values})

    # Calculate percentages for the pie chart
    total_count = counts_df['Count'].sum()
    counts_df['Percentage'] = (counts_df['Count'] / total_count) * 100

    # Group countries with less than 2% into "Others"
    threshold = 2 # Adjust the threshold as needed
    others_df = counts_df[counts_df['Percentage'] < threshold]
    counts_df.loc[counts_df['Percentage'] < threshold, 'Country'] = 'Others'
    counts_df = counts_df.groupby('Country').sum().reset_index()

    # Altair chart for percentage distribution
    pie_chart = alt.Chart(counts_df).mark_bar().encode(
        x='Percentage:Q',
        y=alt.Y('Country:N', sort='-x'),
        color='Country:N',
        tooltip=['Country:N', 'Percentage:Q']
    ).properties(
        width=600,
        height=400,
        title='Country Percentage Distribution'
    )

    # Display the pie chart
    st.altair_chart(pie_chart, use_container_width=True)

    # Display countries and percentages in the 'Others' category horizontally
    # Display subheader with customized font size
    st.markdown("<h3 style='font-size: 14px;'>Countries in 'Others' category (<2%):</h3>", unsafe_allow_html=True)
    st.write(others_df[['Country', 'Percentage']].set_index('Country').T)
else:
    # Display a message when the aggregated DataFrame is empty
    st.write("No data to display.")

2023-09-07 16:27:54.940 
  command:

    streamlit run C:\Users\alexander.medina\Miniconda3\envs\phd2\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
